In [1]:
import pandas as pd
import numpy as np
from src.utilities import get_data_path

# Use the same file name as earlier
file_name = "2025_presidential_round2.csv"
cleaned_path = get_data_path("processed", "poland", f"{file_name[:-4]}_clean.csv")

# Load the cleaned DataFrame
df = pd.read_csv(cleaned_path, sep=";", encoding="utf-8")

print("✅ Cleaned DataFrame loaded.")

✅ Cleaned DataFrame loaded.


In [2]:
pd.set_option('display.max_columns', 100)  # or specify a number instead of None, e.g., 100
# pd.set_option('display.max_rows', 100)          # Control max rows shown
pd.set_option('display.max_rows', None)

In [3]:
# Remove empty postal codes
df = df[df["postal_code"].notna()]

# and possible zagranica:
df = df[df["teryt_gmina"].notna()]

In [4]:
# let's define bucket sizes
MIN_BUCKET_SIZE = 10
MAX_BUCKET_SIZE = 16

In [5]:
# Clean postal codes: remove dash
df['postal_clean'] = df['postal_code'].str.replace('-', '')
df['postal_clean_value_count'] = df['postal_clean'].map(df['postal_clean'].value_counts())

df['postal_2'] = df['postal_clean'].astype(str).str[:2]
df['postal_3'] = df['postal_clean'].astype(str).str[:3]
df['postal_3_value_count'] = df['postal_3'].map(df['postal_3'].value_counts())
df['postal_4'] = df['postal_clean'].astype(str).str[:4]
df['postal_4_value_count'] = df['postal_4'].map(df['postal_4'].value_counts())

In [9]:
# Step 1: Get value counts
postal_3_counts = df['postal_3'].value_counts()

# Step 2: Find postal_3 values with count between 10 and 16
valid_postals = postal_3_counts[(postal_3_counts >= MIN_BUCKET_SIZE) & (postal_3_counts <= MAX_BUCKET_SIZE)].index

# Step 3: Assign 'bucket' column based on valid_postals
df['bucket'] = df['postal_3'].where(df['postal_3'].isin(valid_postals))

In [13]:
# df[df.bucket.notna()].head(100)
# df[df.bucket == "513"]

In [94]:
postal_4_counts = df['postal_4'].value_counts()
valid_postals_4 = postal_4_counts[
    (postal_4_counts >= MIN_BUCKET_SIZE ) & (postal_4_counts <= MAX_BUCKET_SIZE )
].index

df.loc[df['bucket'].isna(), 'bucket'] = df.loc[
    df['postal_4'].isin(valid_postals_4), 'postal_4'
]

In [95]:
# df.head()
# df[df.bucket.notna()]
# df[df.bucket=="5973"]

In [14]:
# Step 4: Extract 4-digit prefix from postal_clean
df['postal_4'] = df['postal_clean'].str[:4]

# Step 5: Work only on rows where bucket is still null and 3-digit group was too big
mask_postal_4 = df['bucket'].isna() & (df['postal_3_value_count'] > MAX_BUCKET_SIZE)

# Step 6: Get value counts for postal_4 within that mask
postal_4_counts = df.loc[mask_postal_4, 'postal_4'].value_counts()

# Step 7: Find postal_4 values with count between 10 and 16
valid_postal_4s = postal_4_counts[(postal_4_counts >= MIN_BUCKET_SIZE) & (postal_4_counts <= MAX_BUCKET_SIZE)].index

# Step 8: Assign bucket for valid postal_4s (only where bucket is still null)
df.loc[mask_postal_4, 'bucket'] = df.loc[mask_postal_4, 'postal_4'].where(
    df.loc[mask_postal_4, 'postal_4'].isin(valid_postal_4s)
)

In [18]:
# df[df.bucket.notna()]
# df[df.bucket == "5973"]

In [97]:
# Step 5: Bucket remaining by postal_clean (postal_5) if count >= 10
postal_clean_counts = df['postal_clean'].value_counts()
valid_postal_5 = postal_clean_counts[postal_clean_counts >= MIN_BUCKET_SIZE].index

mask = df['bucket'].isna() & df['postal_clean'].isin(valid_postal_5)
df.loc[mask, 'bucket'] = df.loc[mask, 'postal_clean']

In [98]:
# df.bucket.head(100)
# df[df.bucket == "58260"]

# df[df.postal_4 == "5826"]

In [19]:
# Step 9: Get 5-digit prefix (full postal code, already cleaned)
df['postal_5'] = df['postal_clean']  # optional, for clarity

# Step 10: Build mask for rows to consider
mask_postal_5 = (
    df['bucket'].isna() &
    (
        (df['postal_3_value_count'] > MAX_BUCKET_SIZE) |
        (df['postal_4'].map(df['postal_4'].value_counts()) > MAX_BUCKET_SIZE)
    )
)

# Step 11: Count postal_5 frequencies only within this mask
postal_5_counts = df.loc[mask_postal_5, 'postal_5'].value_counts()

# Step 12: Find valid postal_5 codes where count is between 10 and 16
valid_postal_5s = postal_5_counts[(postal_5_counts >= MIN_BUCKET_SIZE) & (postal_5_counts <= MAX_BUCKET_SIZE)].index

# Step 13: Assign those postal_5s to bucket
df.loc[mask_postal_5, 'bucket'] = df.loc[mask_postal_5, 'postal_5'].where(
    df.loc[mask_postal_5, 'postal_5'].isin(valid_postal_5s)
)

# Step 14: Preserve oversized groups that share the exact same 5-digit code (don’t split)
for postal_code, count in postal_5_counts.items():
    if count > MAX_BUCKET_SIZE:
        # Check if all rows with this code are still ungrouped
        same_postal_mask = (df['postal_5'] == postal_code) & df['bucket'].isna()
        if same_postal_mask.sum() == count:
            df.loc[same_postal_mask, 'bucket'] = postal_code  # assign full postal_5

In [ ]:
# df[df.bucket.isna()].count()
# df.bucket.value_counts()
# x = df.bucket.nunique()

1179

In [29]:
# Step 15: Get all remaining unassigned polling stations
leftovers = df[df['bucket'].isna()].copy()

# Container for merged leftover groups
new_buckets = []

# Step 16: Group leftovers by postal_3
for prefix, group in leftovers.groupby('postal_3'):
    # Sort by full postal code to ensure continuity
    group_sorted = group.sort_values(by='postal_clean')
    
    # Chunk into groups of 10–16
    i = 0
    while i < len(group_sorted):
        chunk = group_sorted.iloc[i:i+16]
        if len(chunk) >= 10:
            bucket_id = f"{prefix}_L{i//16}"
            df.loc[chunk.index, 'bucket'] = bucket_id
            i += len(chunk)
        else:
            break  # remaining rows too small to form a valid group

In [31]:
# df[df.bucket.isna()].count()

In [32]:
# Step 17: Get final leftovers (still no bucket)
final_leftovers = df[df['bucket'].isna()].copy()

# Group existing bucket sizes for each postal_3 to try merging into them
existing_buckets = df.dropna(subset=['bucket']).copy()
existing_buckets['bucket_size'] = existing_buckets.groupby('bucket')['bucket'].transform('count')

# Try to merge final leftovers into nearby groups
for prefix, group in final_leftovers.groupby('postal_3'):
    group_sorted = group.sort_values(by='postal_clean')
    existing_in_prefix = existing_buckets[
        existing_buckets['postal_3'] == prefix
    ].groupby('bucket').first()

    assigned = False
    for idx, row in group_sorted.iterrows():
        # Try to append to a not-too-large existing group
        for bucket_id, b_row in existing_in_prefix.iterrows():
            current_size = df[df['bucket'] == bucket_id].shape[0]
            if current_size < MAX_BUCKET_SIZE:
                df.at[idx, 'bucket'] = bucket_id
                assigned = True
                break

        # If no spot found, assign a new fallback bucket anyway
        if pd.isna(df.at[idx, 'bucket']):
            fallback_id = f"{prefix}_F{idx}"
            df.at[idx, 'bucket'] = fallback_id

In [38]:
df.bucket.nunique()

3015

In [39]:
# Get value counts per bucket
bucket_sizes = df['bucket'].value_counts()

# Count how many buckets fall into each range
buckets_10_16 = bucket_sizes[(bucket_sizes >= 10) & (bucket_sizes <= 16)].count()
buckets_6_30  = bucket_sizes[(bucket_sizes >= 6) & (bucket_sizes <= 30)].count()

# Optional: total number of buckets
total_buckets = bucket_sizes.count()

# Display results
print(f"Buckets with 10–16 items: {buckets_10_16} ({buckets_10_16 / total_buckets:.1%})")
print(f"Buckets with 6–30 items:  {buckets_6_30} ({buckets_6_30 / total_buckets:.1%})")


Buckets with 10–16 items: 1462 (48.5%)
Buckets with 6–30 items:  1719 (57.0%)
